In [1]:
from pyspark import SparkContext
from geopyspark.geopycontext import GeoPyContext
from geopyspark.geotrellis.catalog import read, read_value, query, write
from geopyspark.geotrellis.constants import SPATIAL, ZOOM, TILE
from geopyspark.geotrellis.geotiff_rdd import get
from geopyspark.geotrellis.rdd import RasterRDD, TiledRasterRDD
from geopyspark.geotrellis.catalog import _construct_catalog, _mapped_cached
from geopyspark.geotrellis.constants import SPATIAL, TILE
from geonotebook.vis.geotrellis.render_methods import render_nlcd, single_band_render_from_color_map
from geonotebook.wrappers import GeoTrellisCatalogLayerData, RddRasterData
from geonotebook.wrappers.vector import GeoJsonData
import numpy as np
from shapely.geometry import Polygon, mapping
from shapely.ops import transform
from functools import partial
import pyproj
from PIL import Image

In [2]:
sc = SparkContext(appName="Potsdam Viz")
geopysc = GeoPyContext(sc)

In [3]:
catalog_uri = "s3://otid-data/viz/catalog"
label_layer_name = "isprs-potsdam-labels"
imagery_layer_name = "isprs-potsdam-imagery-rgb"
rir_layer_name = "isprs-potsdam-imagery-rir"

In [4]:
label_names = ['Impervious',
               'Building',
               'Low vegetation',
               'Tree',
               'Car',
               'Clutter']

label_keys = [
            [255, 255, 255],
            [0, 0, 255],
            [0, 255, 255],
            [0, 255, 0],
            [255, 255, 0],
            [255, 0, 0],
        ]

In [5]:
M.set_center(13.04951140975037, 52.40398631048631, 15)
M.set_zoom_range(0, 22)

In [6]:
def rgb_render(tile):
    alpha = np.full((256, 256), 0xFF)
    alpha[np.ma.where((tile[0] == 0) & (tile[1] == 0) & (tile[2] == 0))] =0x00
    arr = np.dstack([tile[0], tile[1], tile[2], alpha]).astype('uint8')
    return Image.fromarray(arr, mode='RGBA')

def remove_layer(name):
    to_remove = []
    for l in M.layers:
        if l.name == name:
            to_remove.append(l)
    for x in to_remove:
        M.remove_layer(x)
        
def safe_divide(a, b):
    """
    Avoid divide by zero
    http://stackoverflow.com/questions/26248654/numpy-return-0-with-divide-by-zero
    """
    with np.errstate(divide='ignore', invalid='ignore'):
        c = np.true_divide(a, b)
        c[c == np.inf] = 0
        c = np.nan_to_num(c)
        return c


In [7]:
def add_label_layer():
    data = GeoTrellisCatalogLayerData(geopysc, 
                                      catalog_uri, 
                                      label_layer_name,
                                      SPATIAL)
    M.add_layer(data, name="label", render_tile=rgb_render, max_zoom=22)

In [8]:
def add_imagery_layer():
    data = GeoTrellisCatalogLayerData(geopysc, 
                                      catalog_uri, 
                                      imagery_layer_name,
                                      SPATIAL)
    M.add_layer(data, name="aerial", render_tile=rgb_render, max_zoom=22)

In [9]:
def add_ndvi_layer(threshold):
    colors = { 1: "#00FF00FF" }
    cmap_render = single_band_render_from_color_map(colors)
    def render_ndvi(tile):
        rir = tile
        red = rir[0].astype('float64')
        ir = rir[1].astype('float64')
        ndvi = safe_divide((ir - red), (ir + red))
        arr = np.zeros((256,256))
        high_ndvi = np.ma.where(ndvi >= threshold)
        arr[high_ndvi] = 1
        
        return cmap_render(arr)

    data = GeoTrellisCatalogLayerData(geopysc, 
                                      catalog_uri, 
                                      rir_layer_name,
                                      SPATIAL)
    remove_layer("ndvi")
    M.add_layer(data, name="ndvi", render_tile=render_ndvi, max_zoom=22)

In [10]:
def add_diff_layer(model1, model2):
    colors = { 1: "#FF0000FF" }
    cmap_render = single_band_render_from_color_map(colors)
    category = label_names.index('Tree')
    def render_diff(tiles):
        p1, p2 = tiles[0][0], tiles[1][0]
        arr = np.zeros((256,256))
        arr[np.ma.where((p1 != p2) & ((p1 == category) | (p2 == category)))] = 1
        #arr[np.ma.where((p1 != p2) & (labels == label_names.index('Car')))] = 1
        #arr[np.ma.where(p1 != p2)] = 1
        return cmap_render(arr)
    
    model1_layer = 'isprs-potsdam-%s-predictions' % model1
    model2_layer = 'isprs-potsdam-%s-predictions' % model2
    layers = [model1_layer, model2_layer]
    data = GeoTrellisCatalogLayerData(geopysc, 
                                      catalog_uri, 
                                      layers,
                                      SPATIAL)
    remove_layer("diff")
    M.add_layer(data, name="diff", render_tile=render_diff, max_zoom=22)

In [11]:
def add_ndvi_analysis(threshold):
    """
    Layer which indicates what pixels have an NDVI threshold above
    a specific value but are not labeled as low vegetation or tree,
    also pixels that are below the threshold that are labeled
    low vegitation or tree.
    """
    colors = { 1: "#FF0000FF", 2: "#0000FFFF", 3: "#00FF00FF" }
    cmap_render = single_band_render_from_color_map(colors)
    category = label_names.index('Tree')
    def render_tile(tiles):
        rir, labels = tiles[0], tiles[1]
        red = rir[0].astype('float64')
        ir = rir[1].astype('float64')
        ndvi = safe_divide((ir - red), (ir + red))
        arr = np.zeros((256,256))
        
        high_ndvi_no_veg = np.ma.where((ndvi >= threshold) & \
                                   ((labels[0] == 0xFF) | (labels[1] == 0x00)))
        
        low_ndvi_veg = np.ma.where((ndvi < threshold) & \
                                       (labels[0] == 0) & \
                                       (labels[1] == 0xFF))
        
        high_ndvi_veg = np.ma.where((ndvi >= threshold) & \
                                    (labels[0] == 0) & \
                                    (labels[1] == 0xFF))
        arr[high_ndvi_no_veg] = 1
        arr[low_ndvi_veg] = 2
        arr[high_ndvi_veg] = 3
        
        return cmap_render(arr)
    
    layers = [rir_layer_name, label_layer_name]
    data = GeoTrellisCatalogLayerData(geopysc, 
                                      catalog_uri, 
                                      layers,
                                      SPATIAL)
    remove_layer("ndvi-analysis")
    M.add_layer(data, name="ndvi-analysis", render_tile=render_tile, max_zoom=22)

In [12]:
def add_ndvi_tree_analysis(threshold):
    """
    Layer which indicates what pixels have an NDVI threshold above
    a specific value but are not labeled as low vegetation or tree,
    also pixels that are below the threshold that are labeled
    low vegitation or tree.
    """
    colors = { 1: "#FF0000FF", # High NDVI not a tree or low veg [RED]
               2: "#0000FFFF", # Low NDVI and a tree [BLUE]
               3: "#00FF00FF", # High NDVI and a tree [GREEN]
               4: "#91E6F2FF", # Low NDVI and low veg [LIGHT BLUE]
               5: "#77C168FF"  # High NDVI and veg [LIGHT GREEN]
             } 
    cmap_render = single_band_render_from_color_map(colors)
    def render_tile(tiles):
        rir, labels = tiles[0], tiles[1]
        red = rir[0].astype('float64')
        ir = rir[1].astype('float64')
        ndvi = safe_divide((ir - red), (ir + red))
        arr = np.zeros((256,256))
  
        high_ndvi_no_veg = np.ma.where((ndvi >= threshold) & \
                                   ((labels[0] == 0xFF) | 
                                    (labels[1] == 0x00)))
        arr[high_ndvi_no_veg] = 1
        
        low_ndvi_tree = np.ma.where((ndvi < threshold) & \
                                       (labels[0] == 0x00) & \
                                       (labels[1] == 0xFF) & \
                                       (labels[2] == 0x00))
        arr[low_ndvi_tree] = 2
        
        high_ndvi_tree = np.ma.where((ndvi >= threshold) & \
                                    (labels[0] == 0x00) & \
                                    (labels[1] == 0xFF) & \
                                    (labels[2] == 0x00))
        arr[high_ndvi_tree] = 3  
        
        low_ndvi_low_veg = np.ma.where((ndvi < threshold) & \
                                       (labels[0] == 0) & \
                                       (labels[1] == 0xFF) & \
                                       (labels[2] == 0xFF) & \
                                       (arr == 0))

        arr[low_ndvi_low_veg] = 4

        high_ndvi_low_veg = np.ma.where((ndvi >= threshold) & \
                                        (labels[0] == 0x00) & \
                                        (labels[1] == 0xFF) & \
                                        (labels[2] == 0xFF) & \
                                        (arr == 0))
        arr[high_ndvi_low_veg] = 5
        
        return cmap_render(arr)
    
    layers = [rir_layer_name, label_layer_name]
    data = GeoTrellisCatalogLayerData(geopysc, 
                                      catalog_uri, 
                                      layers,
                                      SPATIAL)
    remove_layer("ndvi-analysis-tree")
    M.add_layer(data, name="ndvi-analysis-tree", render_tile=render_tile, max_zoom=22)

In [21]:
#add_label_layer()
#add_imagery_layer()
#add_diff_layer("fcn", "unet")
#add_ndvi_analysis(0.1)
add_ndvi_tree_analysis(0.1)
#add_ndvi_layer(0.3)

::1 - - [2017-05-23 22:54:43] "GET /tile/18/140575/86106.png HTTP/1.1" 200 5974 1.661342
::1 - - [2017-05-23 22:54:43] "GET /tile/18/140576/86106.png HTTP/1.1" 200 8882 0.683118
::1 - - [2017-05-23 22:54:44] "GET /tile/18/140576/86107.png HTTP/1.1" 200 10147 0.721904
::1 - - [2017-05-23 22:54:45] "GET /tile/18/140574/86106.png HTTP/1.1" 200 7614 0.588424
::1 - - [2017-05-23 22:54:45] "GET /tile/18/140575/86108.png HTTP/1.1" 200 9257 0.560605
::1 - - [2017-05-23 22:54:46] "GET /tile/18/140574/86107.png HTTP/1.1" 200 11238 0.578814
::1 - - [2017-05-23 22:54:47] "GET /tile/18/140576/86105.png HTTP/1.1" 200 6407 0.920213
::1 - - [2017-05-23 22:54:47] "GET /tile/18/140575/86105.png HTTP/1.1" 200 7946 0.537804
::1 - - [2017-05-23 22:54:48] "GET /tile/18/140576/86108.png HTTP/1.1" 200 9339 0.618496
::1 - - [2017-05-23 22:54:49] "GET /tile/18/140577/86106.png HTTP/1.1" 200 9272 0.679623
::1 - - [2017-05-23 22:54:49] "GET /tile/18/140577/86107.png HTTP/1.1" 200 9061 0.773451
::1 - - [2017-05-23

In [24]:
add_imagery_layer()

::1 - - [2017-05-22 20:53:47] "GET /tile/15/17571/10763.png HTTP/1.1" 200 111284 1.264673
::1 - - [2017-05-22 20:53:47] "GET /tile/15/17572/10763.png HTTP/1.1" 200 162982 0.345842
::1 - - [2017-05-22 20:53:49] "GET /tile/15/17571/10764.png HTTP/1.1" 200 145057 1.719133
::1 - - [2017-05-22 20:53:49] "GET /tile/15/17572/10764.png HTTP/1.1" 200 157706 0.525453
::1 - - [2017-05-22 20:53:50] "GET /tile/15/17570/10763.png HTTP/1.1" 500 2971 0.150799
::1 - - [2017-05-22 20:53:50] "GET /tile/15/17571/10765.png HTTP/1.1" 200 18284 0.184907
::1 - - [2017-05-22 20:53:50] "GET /tile/15/17570/10764.png HTTP/1.1" 200 92099 0.303480
::1 - - [2017-05-22 20:53:50] "GET /tile/15/17572/10762.png HTTP/1.1" 200 54933 0.154307
::1 - - [2017-05-22 20:53:50] "GET /tile/15/17571/10762.png HTTP/1.1" 200 35348 0.160998
::1 - - [2017-05-22 20:53:51] "GET /tile/15/17572/10765.png HTTP/1.1" 200 15348 0.134071
::1 - - [2017-05-22 20:53:51] "GET /tile/15/17573/10763.png HTTP/1.1" 200 97404 0.329113
::1 - - [2017-05-2

In [ ]:
M.layers

In [22]:
#remove_layer("aerial")
#remove_layer("diff")
#remove_layer("ndvi")
#remove_layer("ndvi-analysis")
remove_layer("ndvi-analysis-tree")
#remove_layer("label")

In [ ]:
M.remove_layer(M.layers[0])

In [ ]:
from geopyspark.geotrellis.catalog import get_layer_ids
layer_ids = list(get_layer_ids(geopysc, catalog_uri))
layer_name = label_layer_name
zooms = []
for lid in layer_ids:
    if lid['name'] == layer_name:
        zooms.append(int(lid['zoom']))
print(min(zooms))
print(max(zooms))

In [ ]:
layer_ids

In [ ]:
l.vis_options.maxZoom


In [ ]:
M.get_map_state()

In [ ]:
M.remove_layer(M.layers[0])

In [ ]:
np.zeros((256,256))

# Analyze FCN vs FCN/DSM performance

In [16]:
from geopyspark.geotrellis import catalog

In [18]:
ZOOM = 20
NUM_PARTITIONS = 500

def tag_and_map(tag):
    def mapper(tup):
        (k, v) = tup
        return ((k['col'], k['row']), (tag, v))
    return mapper

fcn = read(geopysc,
           SPATIAL,
           catalog_uri,
           "isprs-potsdam-fcn-predictions",
           ZOOM, 
           numPartitions=NUM_PARTITIONS).to_numpy_rdd() \
                                        .map(tag_and_map("fcn"))
fcndsm = read(geopysc,
              SPATIAL,
              catalog_uri,
              "isprs-potsdam-fcndsm-predictions",
              ZOOM, 
              numPartitions=NUM_PARTITIONS).to_numpy_rdd() \
                                           .map(tag_and_map("fcndsm"))
labels = read(geopysc,
              SPATIAL,
              catalog_uri,
              label_layer_name,
              ZOOM, 
              numPartitions=NUM_PARTITIONS).to_numpy_rdd() \
                                           .map(tag_and_map("label"))

In [19]:
def label_rgb_to_labels(tile):
    arr = np.zeros((256,256))
    def set_label(idx):
        arr[np.ma.where((tile[0] == label_keys[idx][0]) & \
                        (tile[1] == label_keys[idx][1]) & \
                        (tile[2] == label_keys[idx][2]))] = idx
    for i in range(0, len(label_keys)):
        set_label(i)

    return arr

def mapper(tup):
    v = list(tup[1])
    fcn_tile = next(filter(lambda x: x[0] == "fcn", v))[1]['data'][0]
    fcndsm_tile = next(filter(lambda x: x[0] == "fcndsm", v))[1]['data'][0]
    label_tile = label_rgb_to_labels(next(filter(lambda x: x[0] == "label", v))[1]['data'])
    result = []

    fcn_total_incorrect = 0
    fcndsm_total_incorrect = 0
    
    for i in range(0, len(label_keys)):
        arr = np.zeros((256,256))
        arr[np.ma.where((label_tile != i) & (fcn_tile == i))] = 1
        fcn_false_positives = arr.sum()
        
        arr = np.zeros((256,256))
        arr[np.ma.where((label_tile == i) & (fcn_tile != i))] = 1
        fcn_false_negatives = arr.sum()
        
        fcn_total_incorrect += fcn_false_negatives
        
        arr = np.zeros((256,256))
        arr[np.ma.where((label_tile != i) & (fcndsm_tile == i))] = 1
        fcndsm_false_positives = arr.sum()
        
        arr = np.zeros((256,256))
        arr[np.ma.where((label_tile == i) & (fcndsm_tile != i))] = 1
        fcndsm_false_negatives = arr.sum()
        
        fcndsm_total_incorrect += fcndsm_false_negatives

        result.append((fcn_false_positives, 
                       fcn_false_negatives, 
                       fcndsm_false_positives, 
                       fcndsm_false_negatives))
    return (1, fcn_total_incorrect, fcndsm_total_incorrect, result)
    
def reducer(tup1, tup2):
    (count1, fcn1, fcndsm1, r1) = tup1
    (count2, fcn2, fcndsm2, r2) = tup2
    result = []
    for i in range(0, len(r1)):
        fcn_false_positives = r1[i][0] + r2[i][0]
        fcn_false_negatives = r1[i][1] + r2[i][1]
        fcndsm_false_positives = r1[i][2] + r2[i][2]
        fcndsm_false_negatives = r1[i][3] + r2[i][3]
        result.append((fcn_false_positives, 
                       fcn_false_negatives, 
                       fcndsm_false_positives,
                       fcndsm_false_negatives))
    return (count1 + count2, 
            fcn1 + fcn2,
            fcndsm1 + fcndsm2,
            result)

(tile_count, 
 fcn_total_incorrect,
 fcndsm_total_incorrect,
 result) = sc.union([fcn, fcndsm, labels]) \
             .groupByKey() \
             .map(mapper) \
             .reduce(reducer)


In [20]:
def percent_more(a, b):
    if a > b:
        return ((a - b) / b) * 100
    else:
        return ((b - a) / a) * 100
        
def format_percent(p):
    return "{0:.2f}%".format(p)

total_count = tile_count * 256 * 256
print("TOTAL PIXELS: %d" % total_count)
fcn_wrong_percent = format_percent((fcn_total_incorrect / total_count) * 100)
print("TOTAL FCN INCORRECT: %d (%s)" % (fcn_total_incorrect, fcn_wrong_percent))
fcndsm_wrong_percent = format_percent((fcndsm_total_incorrect / total_count) * 100)
print("TOTAL FCN w/ DSM INCORRECT: %d (%s)" % (fcndsm_total_incorrect, fcndsm_wrong_percent))
for i, (fcn_false_positives, 
        fcn_false_negatives, 
        fcndsm_false_positives, 
        fcndsm_false_negatives) in enumerate(result):
    print("%s: " % label_names[i])
    print("\tFalse Positives: ")
    print("\t\tFCN: %d\tFCN w/ DSM: %d" % (fcn_false_positives, fcndsm_false_positives))
    dp = fcn_false_positives - fcndsm_false_positives
    p = format_percent(percent_more(fcn_false_positives, fcndsm_false_positives))
    if dp < 0:
        print("\t\tFCN DSM got %d more false positives. (%s)" % (-dp, p))
    else:
        print("\t\tFCN got %d more false positives. (%s)" % (dp, p))

    print("\tFalse Negatives: ")
    print("\t\tFCN: %d\tFCN w/ DSM: %d" % (fcn_false_negatives, fcndsm_false_negatives))
    dn = fcn_false_negatives - fcndsm_false_negatives
    n = format_percent(percent_more(fcn_false_negatives, fcndsm_false_negatives))
    if dn < 0:
        print("\t\tFCN DSM got %d more false negatives. (%s)" % (-dn, n))
    else:
        print("\t\tFCN got %d more false negatives. (%s)" % (dn, n))
   

TOTAL PIXELS: 274006016
TOTAL FCN INCORRECT: 49475435 (18.06%)
TOTAL FCN w/ DSM INCORRECT: 50995123 (18.61%)
Impervious: 
	False Positives: 
		FCN: 8900965	FCN w/ DSM: 8219286
		FCN got 681679 more false positives. (8.29%)
	False Negatives: 
		FCN: 21960638	FCN w/ DSM: 23347061
		FCN DSM got 1386423 more false negatives. (6.31%)
Building: 
	False Positives: 
		FCN: 5111063	FCN w/ DSM: 5025113
		FCN got 85950 more false positives. (1.71%)
	False Negatives: 
		FCN: 4069051	FCN w/ DSM: 3318913
		FCN got 750138 more false negatives. (22.60%)
Low vegetation: 
	False Positives: 
		FCN: 10930413	FCN w/ DSM: 12173261
		FCN DSM got 1242848 more false positives. (11.37%)
	False Negatives: 
		FCN: 10441264	FCN w/ DSM: 9949842
		FCN got 491422 more false negatives. (4.94%)
Tree: 
	False Positives: 
		FCN: 6585154	FCN w/ DSM: 7196733
		FCN DSM got 611579 more false positives. (9.29%)
	False Negatives: 
		FCN: 6630455	FCN w/ DSM: 8167376
		FCN DSM got 1536921 more false negatives. (23.18%)
Car: 
	Fa

In [34]:
result

[(4786.0, 425733.0, 4421.0, 426509.0),
 (29725.0, 5012.0, 28958.0, 4368.0),
 (21530.0, 15854.0, 21770.0, 14792.0),
 (9738.0, 10598.0, 9430.0, 11701.0),
 (2339.0, 4224.0, 2400.0, 4214.0),
 (1006.0, 7861.0, 1658.0, 7211.0)]